In [1]:
import pandas as pd
from functools import reduce
import numpy as np
from tika import parser
import requests
import utm


import sys

sys.path.append("..")
from tools import data_path, output_path, cache_path

### Data origin: 

- Data was privately provided by authorities. 
- Not reproducible without original data.
- This is going to be a wild one :) 6 different formats.
- Useful: chardet.detect(f)

In [2]:
path = data_path + "/mv/vision_group_jena(1)/"

In [3]:
from os import listdir
from os.path import isfile, join
mypath = path
onlyfiles = [mypath + f for f in listdir(mypath) if not isfile(join(mypath, f))]

In [4]:
def format_1(which, column_names = ["ID", "?", "date", "time", "value"], encoding=None, remove_quotation= True):
    # open and parse to df    
    print(which)
    print(column_names)
    if not encoding:
        f = open(which, "r")
    else:
        f = open(which, "r", encoding = encoding)

    f = f.read()
    f = f.split("\n")
    f = [x.split(";") for x in f]
    f = pd.DataFrame(f[:-1])
    f.columns = column_names
    # Check
    #check if the columns have unique ids and values
    assert len(f["?"].unique()) == 1, "Issue, nonunique" 
    assert len(f["ID"].unique()) == 1, "Issue nonunique"
    #reformat values to float and convert timestamps to ds
    f.drop(index=f[f["value"] == "---"].index, inplace=True)
    f[f["ID"][0]] = f["value"].str.replace(",",".").astype(float)
    if remove_quotation:
        f.index = pd.to_datetime(f["date"].astype(str).str[1:-1] + " " + f["time"].astype(str).str[1:-1], format="%d.%m.%Y %H:%M:%S")
    else:
        if "time" in f.columns:
            f.index = pd.to_datetime(f["date"].astype(str) + " " + f["time"].astype(str), format="%d.%m.%Y %H:%M:%S")
        else:
            f.index = pd.to_datetime(f["date"].astype(str) , format="%d.%m.%Y")
    f = f[[f["ID"][0]]]
    return f


In [5]:
def format_2(which):
    print(which)
    name = pd.read_excel(which, nrows=1).columns[0]
    f = pd.read_excel(which, header=2)
    for x in range(16):
        f.loc[f["Q in m³/s"] == "[" + str(x) + "]", "Q in m³/s"] = np.nan
    if "Datum " in f.columns:
        f.index = pd.to_datetime(f["Datum "].astype(str) + " " + f["Uhrzeit"].astype(str),  format="%d.%m.%Y %H:%M:%S")
    else:
        f.index = pd.to_datetime(f["Datum"].astype(str) + " " + f["Uhrzeit"].astype(str),  format="%d.%m.%Y %H:%M:%S")

    f[name] = f["Q in m³/s"].astype(float)
    return f[[name]]

In [6]:
def format_3(which):
    print(which)
    f = pd.read_csv(which, sep=";", header=None, skiprows=1 if "Wehrland" in which else 0)
    f.index = pd.to_datetime(f[0].astype(str) + " " + f[1],format= "%Y%m%d %H:%M:%S" )
    f = f[[2]]
    f.columns = [which.split("/")[-1].split("_")[0]]
    return f

In [7]:
def format_4(which, column_names = ["date", "time", "value", "?"], encoding="ISO-8859-1"):
    # open and parse to df    
    print(which)
    if not encoding:
        f = open(which, "r")
    else:
        f = open(which, "r", encoding = encoding)
    f = f.read()
    f = f.split("\n")
    f = [x.split("\t") for x in f]
    f = pd.DataFrame(f[:-1])
    f.columns = column_names
    # Check
    #check if the columns have unique ids and values
    assert len(f["?"].unique()) == 1, "Issue, nonunique" 
    #reformat values to float and convert timestamps to ds
    f.drop(index=f[f["value"] == "---"].index, inplace=True)
    f[which.split("/")[-1][2:-4]] = f["value"].str.replace(",",".").astype(float)
    f.index = pd.to_datetime(f["date"].astype(str) + " " + f["time"].astype(str), format="%d.%m.%Y %H:%M")
    f = f.drop(columns=["date", "time", "?", "value"])
    return f

In [8]:
stack1 = listdir(onlyfiles[0] + "/" + listdir(onlyfiles[0])[0])
stack1 = [onlyfiles[0] + "/" + listdir(onlyfiles[0])[0] + "/" + x for x in stack1]
stack1 = [format_1(x)  for x in stack1]
stack1 = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), stack1)
stack1.columns = ["0" + str(int(x)) for x in stack1.columns]
stack1 = stack1.resample("15T").mean()

/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MS/Zuarbeit StALU MS/Klempenow_Q_2019-2024.txt
['ID', '?', 'date', 'time', 'value']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MS/Zuarbeit StALU MS/Brohm_Q_OP_2019-2024.txt
['ID', '?', 'date', 'time', 'value']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MS/Zuarbeit StALU MS/Burg-Stargard_ Q_2019-2024.txt
['ID', '?', 'date', 'time', 'value']


...

/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MS/Zuarbeit StALU MS/Neukalen_Q_2019-2024.txt
['ID', '?', 'date', 'time', 'value']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MS/Zuarbeit StALU MS/Kittendorf_Q_2019-2024.txt
['ID', '?', 'date', 'time', 'value']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]
/tmp/ipykernel_241635/4044109703.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  stack1 = stack1.resample("15T").mean()


In [9]:
stack2 = listdir(onlyfiles[1] + "/" + listdir(onlyfiles[1])[0])
stack2 = [onlyfiles[1] + "/" + listdir(onlyfiles[1])[0] + "/" + x for x in stack2]
stack2 = [format_1(x, column_names = ["ID", "date", "time", "value", "?"], encoding= "ISO-8859-1", remove_quotation=False)  for x in stack2]
stack2 = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), stack2)
stack2.columns = ["0" + str(int(x)) for x in stack2.columns]
stack2 = stack2.resample("15T").mean()

/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU WM/Daten/59618.0_Kronskamp_Q_3_1_2.txt
['ID', 'date', 'time', 'value', '?']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU WM/Daten/04381.0_BarnerStück_Q_3_1_2.txt
['ID', 'date', 'time', 'value', '?']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU WM/Daten/59905.0_Schwartow_Q_3_1_2.txt
['ID', 'date', 'time', 'value', '?']


...

/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU WM/Daten/59870.1_Hagenow_Q_3_1_2.txt
['ID', 'date', 'time', 'value', '?']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU WM/Daten/04413.0_Rönkenhof_Q_3_1_2.txt
['ID', 'date', 'time', 'value', '?']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]
/tmp/ipykernel_241635/1457206842.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  stack2 = stack2.resample("15T").mean()


In [10]:
stack3 = [onlyfiles[2] +"/" +  x for x in listdir(onlyfiles[2])]
stack3 = [format_2(x)  for x in stack3]
stack3 = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), stack3)

/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MM/2024-07-03 15Min_Teßmannsdorf_Hellbach 2019-2024.xlsx
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MM/2024-07-03 5Min_Rostock-Geinitzbrücke_Warnow 2019-2024.xlsx
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MM/2024-07-03 15Min_Dobbin_Nebel 2019-2024.xlsx
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MM/2024-07-03 15Min_BützowGesamt_Warnow 2019-2024.xlsx
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MM/2024-07-03 15Min_Tessin_Recknitz 2019-2024.xlsx
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MM/2024-07-03 15Min_Wolken_Nebel 2019-2024.xlsx
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU MM/2024-07-03 15Min_Schwaan_Beke 2019-2024.xlsx


In [11]:
# quickly check the ids via: https://pegelportal-mv.de/pegel-mv/pegel_list.html
ids = ["043902", "044022", "044431","044051", "045130","044290","044253"]
stack3.columns = ids
stack3 = stack3.resample("15T").mean()

/tmp/ipykernel_241635/2567875862.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  stack3 = stack3.resample("15T").mean()


In [12]:
stack4 = [onlyfiles[3] +"/" +  x for x in listdir(onlyfiles[3])]
stack4 = [format_3(x) for x in stack4]
stack4 = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), stack4)

/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM West/47015_Libnow.csv
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM West/47023_Anklam.csv
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM West/47402_GützkowHasenberg.csv
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM West/46641_Lodmannshagen.csv
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM West/46614_Hanshagen.csv
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM West/47453_Schwinge.csv
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM West/47352_Padderow.csv
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM West/46660_Wehrland.csv


In [13]:
stack4.columns = ["0" + x for x in stack4.columns ]

In [14]:
stack4 = stack4.replace("---/", np.nan).astype(float)

In [15]:
stack4 = stack4.resample("15T").mean()

/tmp/ipykernel_241635/4168897223.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  stack4 = stack4.resample("15T").mean()


In [16]:
stack5 = [onlyfiles[4] +"/" +  x for x in listdir(onlyfiles[4])]
stack5 = [format_4(x) for x in stack5]
stack5 = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), stack5)
stack5 = stack5.resample("15T").mean()

/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_047590.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_045500.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_047562.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_045590.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_046111.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_046401.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_045061.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_046161.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_047700.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_045411.txt
/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP_HST/Q_046175.txt
/home/datasets4/stein

/tmp/ipykernel_241635/2161987191.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  stack5 = stack5.resample("15T").mean()


In [17]:
stack6 = [onlyfiles[5] +"/" +  x for x in listdir(onlyfiles[5])]
stack6 = [
    format_1(
        x,
        encoding="ISO-8859-1",
        column_names=(
            ["ID", "date", "value", "?"]
            if "Bollwerk" in x
            else ["ID", "date", "time", "value", "??", "?"]
        ),
        remove_quotation=False,
    )
    for x in stack6
]

# seems broken as we cant recover timestamps properly. leave out for now.
stack6[2].index.value_counts().unique()
del stack6[2]
stack6 = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), stack6)

/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM_Ost/049012_Ueckermünde-Klappbrücke_Q.txt
['ID', 'date', 'time', 'value', '??', '?']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM_Ost/049301_Rieth_Q.txt
['ID', 'date', 'time', 'value', '??', '?']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM_Ost/049071_Pasewalk-Bollwerk_Q.txt
['ID', 'date', 'value', '?']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM_Ost/048602_Gehren-Wald_Q.txt
['ID', 'date', 'time', 'value', '??', '?']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


/home/datasets4/stein/rivers/raw_data//mv/vision_group_jena(1)/StALU VP UEM_Ost/048601_Heinrichswalde-Nord_Q.txt
['ID', 'date', 'time', 'value', '??', '?']


/tmp/ipykernel_241635/2475564141.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = f[[f["ID"][0]]]


In [18]:
stack6.columns = ["0" + str(int(x)) for x in stack6.columns]
stack6 = stack6.resample("15T").mean()

/tmp/ipykernel_241635/1000634505.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  stack6 = stack6.resample("15T").mean()


In [19]:
'2021-03-04 12:02:00' in stack6.index

False

In [20]:
data_product = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,
                                            how='outer'), [stack1,stack2,stack3,stack4,stack5,stack6])


In [21]:
# mandatory consistency check. There shouldnt be any double ids.
len(data_product.columns) == len(set(data_product.columns.values))

True

In [22]:
data_product.to_csv(cache_path + "mv_cache.csv")

In [23]:
# get all info from files to base meta data on. merge everything and be happy. 
# Also fix the resolution thing directly when parsing to reduce processing time.

In [24]:
# according to the website only ids with 4 have leading zeros.
data_product.columns = [x[1:] if x[1] == "5" else x for x in data_product.columns.values]

In [25]:
fail = []
for x in data_product.columns:
    try:
        url_base = "https://pegelportal-mv.de/pdf/pegelsteckbrief_"
        path = "/home/datasets4/stein/rivers/raw_data/mv/pdf/pegelsteckbrief_"
        url = url_base + str(x)[:-1] + "." + str(x[-1]) + ".pdf"
        save_path = path + str(x)[:-1] + "." + str(x[-1]) + ".pdf"

        response = requests.get( url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
        else:
            fail.append(x)
    except:
        fail.append(x)

In [26]:
data_product.drop(columns = fail, inplace=True)

In [27]:
data_product = data_product[data_product.index.year < 2024]

In [28]:
meta = []
for x in data_product.columns: 
    raw = parser.from_file(data_path + "/mv/pdf/pegelsteckbrief_" + x[:-1] +  "." + x[-1] + ".pdf")
    X= int(raw["content"].split("Rechtswert (ETRS89)")[1].split("\n")[0][3:])
    Y= int(raw["content"].split("Hochwert (ETRS89)")[1].split("\n")[0])
    Z = float(raw["content"].split("Pegelnull / System")[1].split("\n")[0].split("/")[0].replace(",","."))
    R = raw["content"].split("Gewässer")[1].split("\n")[0].strip(" ")
    meta.append([R,X,Y,np.nan,Z])

In [29]:
meta = pd.DataFrame(meta,index=data_product.columns, columns = ["R", "X", "Y","D", "H"])

In [30]:
# saxony parsing
for ind, line in meta.iterrows():
    meta.loc[ind, ["X", "Y"]] = utm.to_latlon(
        line["X"], line["Y"], 33, "U"
    )

/tmp/ipykernel_241635/3535649114.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '53.79228781192966' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  meta.loc[ind, ["X", "Y"]] = utm.to_latlon(
/tmp/ipykernel_241635/3535649114.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.311800586833666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  meta.loc[ind, ["X", "Y"]] = utm.to_latlon(


In [31]:
meta["QD"] = -1
meta["QH"] = -1
meta["QX"] = -1
meta["QY"] = -1
meta["QR"] = -1

In [32]:
meta.loc[~(meta["X"].isnull()), "QX"] = 0
meta.loc[~(meta["Y"].isnull()), "QY"] = 0
meta.loc[~(meta["D"].isnull()), "QD"] = 0
meta.loc[~(meta["R"].isnull()), "QR"] = 0
meta.loc[~(meta["H"].isnull()), "QH"] = 0

In [33]:
# Important! During a late stage of the project, MV updated there data sites, which allowed us to actually download meta data on 8 more stations which we previously dropped. 
# To not disturb the already gained results, we will still drop them here.

drop_to_confirm_previous_state = [
        '595890',
        '595005',
        '598701',
        '047402',
        '046641',
        '046614',
        '047453',
        '046660'
 ]

In [34]:
meta.drop(index=drop_to_confirm_previous_state, inplace=True)

In [35]:
data_product.columns = [x + "_mv" for x in data_product.columns]


In [41]:
data_product.drop(columns= [x + "_mv" for x in drop_to_confirm_previous_state], inplace=True)

In [43]:
data_product.to_csv(output_path + "mv_processed.csv")

In [44]:
meta.to_csv(output_path + "mv_meta_data.csv")
